# Lectura de archivos

In [6]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
id = '1br3fMwXX_J0XmiXvOm_wfKWvHSj45T3y'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_values.csv')

train_values = pd.read_csv('train_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'}, index_col='building_id')

In [9]:
id = '1RUtolRcQlR3RGULttM4ZoQaK_Ouow4gc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_labels.csv')
train_labels = pd.read_csv('train_labels.csv', encoding='latin-1', dtype={'building_id': 'int64', 'damage_grade': 'int64'}, index_col='building_id')

In [10]:
print(train_labels.shape)
print(train_values.shape)

(260601, 1)
(260601, 38)


In [11]:
id = '1kt2VFhgpfRS72wtBOBy1KDat9LanfMZU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_values.csv')

test_values = pd.read_csv('test_values.csv', encoding='latin-1', dtype = {'building_id': 'int64', 'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',\
                                                                           'count_floors_pre_eq': 'int64', 'age': 'int64', 'area_percentage': 'int64', \
                                                                           'height_percentage': 'int64', 'land_surface_condition': 'category',\
                                                                           'foundation_type': 'category', 'roof_type': 'category', 'ground_floor_type': 'category',\
                                                                           'other_floor_type': 'category', 'position': 'category', 'plan_configuration': 'category',\
                                                                           'has_superstructure_adobe_mud': 'boolean', 'has_superstructure_mud_mortar_stone': 'boolean', \
                                                                           'has_superstructure_stone_flag': 'boolean', 'has_superstructure_cement_mortar_stone': 'boolean',\
                                                                           'has_superstructure_mud_mortar_brick': 'boolean', 'has_superstructure_cement_mortar_brick': 'boolean',\
                                                                           'has_superstructure_timber': 'boolean', 'has_superstructure_bamboo': 'boolean',\
                                                                           'has_superstructure_rc_non_engineered': 'boolean', 'has_superstructure_rc_engineered': 'boolean',\
                                                                           'has_superstructure_other': 'boolean', 'legal_ownership_status': 'category', 'count_families': 'int64', \
                                                                           'has_secondary_use': 'boolean', 'has_secondary_use_agriculture': 'boolean', 'has_secondary_use_hotel': 'boolean', \
                                                                           'has_secondary_use_rental': 'boolean', 'has_secondary_use_institution': 'boolean', 'has_secondary_use_school': 'boolean',\
                                                                           'has_secondary_use_industry': 'boolean', 'has_secondary_use_health_post': 'boolean', \
                                                                           'has_secondary_use_gov_office': 'boolean', 'has_secondary_use_use_police': 'boolean', 'has_secondary_use_other': 'boolean'})

In [12]:
test_values.set_index('building_id', inplace=True)

In [13]:
train_values_original = train_values
test_values_original = test_values

# Feature Engineering

## One-Hot Encoding de variable categóricas

In [14]:
one_hot_train = pd.DataFrame({'building_id': train_values.index.tolist()})
one_hot_train.set_index('building_id', inplace=True)

one_hot_test = pd.DataFrame({'building_id': test_values.index.tolist()})
one_hot_test.set_index('building_id', inplace=True)

In [15]:
from sklearn.preprocessing import OneHotEncoder

def encoding(column_name, df):
  df_filt = df[column_name] # nos quedamos con esos valores

  encoder = OneHotEncoder(sparse=False) # armamos el encoder. sparse=False nos devuelve una matriz comun a la que estamos acostumbrados (en vez de esparsa)
  direction = np.array(df_filt).reshape(-1, 1) # aca pasamos a un vector la serie de pandas, como es usual
  encoder.fit(direction) # fiteo
  direction_hot = encoder.transform(direction) # obtenemos la mariz binaria

  return encoder, direction_hot

In [16]:
def one_hot_encoding(column_name, df, one_hot):
  col = df[column_name]

  encoder, direction_hot = encoding(column_name, df)

  new_cat = []
  for i in range(len(encoder.categories_[0])):
    new_cat.append([])

  for categorical_value in range(len(direction_hot)):
    for i in range(len(encoder.categories_[0])):
      new_cat[i].append(direction_hot[categorical_value][i])

  for i, cat in enumerate(encoder.categories_[0]):
    col_name = column_name + '_' + cat
    col_value = new_cat[i]

    one_hot[col_name] = col_value

  return one_hot

In [17]:
def binary_feature(df, df_aux):

  binary_feature = df.select_dtypes('category').columns.tolist()

  for feature in binary_feature:
    df_aux = one_hot_encoding(feature, df, df_aux)
  
  for feature in binary_feature:
    del df[feature]

binary_feature(train_values, one_hot_train)
binary_feature(test_values, one_hot_test)

In [18]:
train_values = train_values.merge(one_hot_train, on='building_id')
train_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,True,True,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
test_values = test_values.merge(one_hot_test, on='building_id')
test_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,False,True,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
def boolean_feature(df):

  boolean_feature = df.select_dtypes('boolean').columns.tolist()

  for feature in boolean_feature:
    df[feature] = df[feature].astype('int64')

boolean_feature(train_values)
boolean_feature(test_values)

In [21]:
train_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
test_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Impacto en su zona geográfica:

###Promedio de edad según la zona geográfica:

In [20]:
def avg_age(df_values):
  geo_levels = df_values[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age']]
  geo_levels.reset_index(inplace=True)

  age_avg_1 = geo_levels.groupby('geo_level_1_id')['age'].mean()
  age_avg_2 = geo_levels.groupby('geo_level_2_id')['age'].mean()
  age_avg_3 = geo_levels.groupby('geo_level_3_id')['age'].mean()

  age_avg_1.rename('age_avg_geo_level_1_id', inplace=True)
  age_avg_2.rename('age_avg_geo_level_2_id', inplace=True)
  age_avg_3.rename('age_avg_geo_level_3_id', inplace=True)

  geo_levels = geo_levels\
        .merge(age_avg_1, on='geo_level_1_id')\
        .merge(age_avg_2, on='geo_level_2_id')\
        .merge(age_avg_3, on='geo_level_3_id')

  geo_levels.set_index('building_id', inplace=True)

  new_features = ['building_id', 'age_avg_geo_level_1_id', 'age_avg_geo_level_2_id', 'age_avg_geo_level_3_id']

  geo_levels.reset_index(inplace=True)
  df_values.reset_index(inplace=True)

  df_values = df_values.merge(geo_levels[new_features]).set_index('building_id')
  return df_values

In [21]:
train_values = avg_age(train_values)
train_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33.021205,22.203704,23.378378


In [22]:
test_values = avg_age(test_values)
test_values.head(1)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_m,ground_floor_type_v,ground_floor_type_x,ground_floor_type_z,other_floor_type_j,other_floor_type_q,other_floor_type_s,other_floor_type_x,position_j,position_o,position_s,position_t,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,age_avg_geo_level_1_id,age_avg_geo_level_2_id,age_avg_geo_level_3_id
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,22.799919,24.666667,33.75


#Feature Selection

###Método 1:

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

In [24]:
print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)

(260601, 68)
(260601, 1)
(86868, 68)


In [25]:
scaler = MinMaxScaler()
scaler.fit(train_values)
x_train = scaler.transform(train_values)

Para el método SelectKBest las métricas posibles para clasificación están en: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

Elegí chi2 porque es la más usada, no sé qué tan bien está porque usamos otra métrica, pero no me deja ponerla.

Creo que estaría bueno hacerlo para las 3 métricas que figuran ahí, y quedarnos con la unión de las 3 métricas (mutual_info_classif está piola porque mide la dependencia entre los features) -> Hasta hay más formas de calcularlos.

La conclusión es que habría o que ver qué forma de calcularlos nos conviene más, o hacer varias e ir quedándonos con intersección o unión de todo eso que calculamos.

In [26]:
selector = SelectKBest(chi2, 30)
selector.fit(x_train, train_labels)

SelectKBest(k=30, score_func=<function chi2 at 0x7f3a93fd6050>)

In [27]:
x_train_fs = selector.transform(x_train)

In [28]:
k_features = np.array(selector.get_support()).reshape((1,-1))
k_features

array([[ True, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,
        False,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True, False,  True, False,
        False,  True,  True,  True, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False,
         True,  True, False, False, False]])

In [29]:
selected_features = []
features = train_values.columns.tolist()
for i, feature in enumerate(features):
  if k_features[0][i]:
    selected_features.append(feature)

selected_features

['geo_level_1_id',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'foundation_type_i',
 'foundation_type_r',
 'foundation_type_u',
 'foundation_type_w',
 'roof_type_n',
 'roof_type_q',
 'roof_type_x',
 'ground_floor_type_f',
 'ground_floor_type_v',
 'other_floor_type_j',
 'other_floor_type_q',
 'other_floor_type_s',
 'position_t',
 'plan_configuration_u',
 'legal_ownership_status_a']

In [30]:
train_values_selected_features = train_values[selected_features]
train_values_selected_features.head()

,geo_level_1_id,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_v,other_floor_type_j,other_floor_type_q,other_floor_type_s,position_t,plan_configuration_u,legal_ownership_status_a
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
28830,8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
94947,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
590882,22,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201944,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
test_values_selected_features = test_values[selected_features]
test_values_selected_features.head()

,geo_level_1_id,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,ground_floor_type_v,other_floor_type_j,other_floor_type_q,other_floor_type_s,position_t,plan_configuration_u,legal_ownership_status_a
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
99355,6,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
890251,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
745817,26,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
421793,17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


###Método 2:
NO CORRER

In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

Jugar un poco con los parámetros

In [25]:
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='f1_micro',
              min_features_to_select=15)

In [ ]:
rfecv.fit(x_train, train_labels)

In [ ]:
rfecv.n_features_

# Pasaje a CSV de los nuevos Features

In [33]:
train_values_selected_features.to_csv('train_values_selected_features.csv')

In [34]:
test_values_selected_features.to_csv('test_values_selected_features.csv')